## Dependencies

In [1]:
# Import machine learning dependencies

import tensorflow as tf
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
# Import other dependencies

from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import pickle

In [3]:
# Python SQL toolkit and Object Relational Mapper

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

## Import Pickled File - From Anu's connection to SQL

In [4]:
# Open pickle
df = pickle.load(open('Resources/final_pickle.pkl', 'rb'))

In [5]:
df.head()

,loan_amount_000s,msamd,county_code,census_tract_number,applicant_income_000s,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,...,co_applicant_sex_Female,co_applicant_sex_Male,co_applicant_sex_No Co-Applicant,Region_MA,Region_MW,Region_NE,Region_NW,Region_SE,Region_SW,Region_W
11,92.0,33860.0,1.0,201.0,29.0,1948.0,12.58,59700.0,122.93,507.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
13,94.0,33860.0,1.0,201.0,53.0,1948.0,12.58,59700.0,122.93,507.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
18,147.0,33860.0,1.0,201.0,90.0,1948.0,12.58,59700.0,122.93,507.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
24,115.0,33860.0,1.0,201.0,30.0,1948.0,12.58,59700.0,122.93,507.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
33,160.0,33860.0,1.0,201.0,50.0,1948.0,12.58,59700.0,122.93,507.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [6]:
# Check the columns again
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168940 entries, 11 to 1700351
Data columns (total 59 columns):
 #   Column                                                         Non-Null Count   Dtype  
---  ------                                                         --------------   -----  
 0   loan_amount_000s                                               168940 non-null  float64
 1   msamd                                                          168940 non-null  float64
 2   county_code                                                    168940 non-null  float64
 3   census_tract_number                                            168940 non-null  float64
 4   applicant_income_000s                                          168940 non-null  float64
 5   population                                                     168940 non-null  float64
 6   minority_population                                            168940 non-null  float64
 7   hud_median_family_income                     

In [7]:
# Remove repetitive columns
df = df.drop(columns = ["owner_occupancy_Not Owner Occupied", "preapproval_Preapproval Not Requested", "applicant_ethnicity_Not Hispanic or Latino", ])
df.head()

,loan_amount_000s,msamd,county_code,census_tract_number,applicant_income_000s,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,...,co_applicant_sex_Female,co_applicant_sex_Male,co_applicant_sex_No Co-Applicant,Region_MA,Region_MW,Region_NE,Region_NW,Region_SE,Region_SW,Region_W
11,92.0,33860.0,1.0,201.0,29.0,1948.0,12.58,59700.0,122.93,507.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
13,94.0,33860.0,1.0,201.0,53.0,1948.0,12.58,59700.0,122.93,507.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
18,147.0,33860.0,1.0,201.0,90.0,1948.0,12.58,59700.0,122.93,507.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
24,115.0,33860.0,1.0,201.0,30.0,1948.0,12.58,59700.0,122.93,507.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
33,160.0,33860.0,1.0,201.0,50.0,1948.0,12.58,59700.0,122.93,507.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Run Neural Network

In [8]:
# Split our preprocessed data into our features and target arrays
y = df["action_taken_summary"].values
X = df.drop(["action_taken_summary"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

c:\Users\olivi\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
import keras

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 56
hidden_nodes_layer2 = 38
hidden_nodes_layer3 = 26

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

## Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation=keras.layers.LeakyReLU(alpha=0.05)))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 56)                3136      
                                                                 
 dense_1 (Dense)             (None, 38)                2166      
                                                                 
 dense_2 (Dense)             (None, 26)                1014      
                                                                 
 dense_3 (Dense)             (None, 1)                 27        
                                                                 
Total params: 6,343
Trainable params: 6,343
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=50)

Epoch 1/50
3960/3960 [==============================] - 13s 3ms/step - loss: 15.3233 - accuracy: 0.7782
Epoch 2/50
3960/3960 [==============================] - 11s 3ms/step - loss: 1.1998 - accuracy: 0.8098
Epoch 3/50
3960/3960 [==============================] - 11s 3ms/step - loss: 0.4835 - accuracy: 0.8639
Epoch 4/50
3960/3960 [==============================] - 11s 3ms/step - loss: 0.3882 - accuracy: 0.8714
Epoch 5/50
3960/3960 [==============================] - 11s 3ms/step - loss: 0.3837 - accuracy: 0.8715
Epoch 6/50
3960/3960 [==============================] - 11s 3ms/step - loss: 0.3837 - accuracy: 0.8715
Epoch 7/50
3960/3960 [==============================] - 11s 3ms/step - loss: 0.3837 - accuracy: 0.8715
Epoch 8/50
3960/3960 [==============================] - 11s 3ms/step - loss: 0.3837 - accuracy: 0.8715
Epoch 9/50
3960/3960 [==============================] - 11s 3ms/step - loss: 0.3837 - accuracy: 0.8715
Epoch 10/50
3960/3960 [==============================] - 11s 3ms/step - 

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1320/1320 - 3s - loss: 0.3965 - accuracy: 0.8694 - 3s/epoch - 2ms/step
Loss: 0.396471232175827, Accuracy: 0.8693500757217407


NameError: name 'model' is not defined